# Exercício de Treinamento para Memória Virtual

Este colab tem 5 exercícios para cálculo do espaço ocupado pela tabela de páginas para memória virtual com 1 ou 2 niveis. Tem exercícios com a situação ótima de ocupação das molduras com dois níveis considerando que o segundo nível usa apenas 1 moldura. O tamanho dos ponteiros pode ser de 1, 2 ou 4 bytes, dependendo da especificações do problema (virtual, física e página). Tem um exercício de mapeamento de endereço.

[Consulte também o Colab com Introdução a Memória Virtual](https://colab.research.google.com/drive/1yFMBKIOR94sudsBO0Z3nhZ4xExBITQnx?usp=sharing)


In [ ]:
#@title Instalação das Bibliotecas
!pip install paho-mqtt &> /dev/null
# !pip install simpy &> /dev/null
ID =  input("Digite a sua matrícula (somente números, sem espaços) ")
topic =  "inf450/2023/Colab_Virtual/"+str(ID)
from datetime import datetime
def getCurrentTime():
  myobj = datetime.now()
  t = str(myobj.hour)+"_"+str(myobj.minute)+"_"+str(myobj.second)
  return t

t = getCurrentTime()

Digite a sua matrícula (somente números, sem espaços) 71269


In [ ]:
#@title funcoes de apoio
from paho.mqtt import client as mqtt_client


def publish(client, pergunta, resposta, topico, initialTime,responseTime):
  form_pergunta = pergunta.replace('\n',"").replace(',',"")
  msg = f"{form_pergunta}; {resposta}; {topico}; {initialTime}; {responseTime}; {t}"
  print(t)
  top = topic+"/"+t
  result = client.publish(top, msg)
  status = result[0]
  if status == 0:
            print(f"Send `{msg}` to topic `{top}`")
  else:
            print(f"Failed to send message to topic {top}")
  client.disconnect() # disconnect gracefully
  client.loop_stop() # stops network loop

def connect_mqtt():
    print("try to connect")
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("Conectou no MQTT Broker!")
        else:
            print("Falhou a conexão com o Broker, return code %d\n", rc)

    client = mqtt_client.Client(client_id)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client

import random
import time



broker = 'broker.emqx.io'
port = 1883
# generate client ID with pub prefix randomly
client_id = ID+f'Ex_virtual{random.randint(0, 1000)}'

print(client_id)

def monitora(pergunta,resposta,id,initialTime,responseTime):
  client = connect_mqtt()
  client.loop_start()
  publish(client,pergunta,resposta,id, initialTime, responseTime)
  client.loop_stop()


71269Ex_virtual89


In [ ]:
#@title Classe Auxiliar
#Produz dropdown de potencias de 2 e um botao, executa a funcao f, quando o botao e clicado
import ipywidgets as widgets
import math

Table = [2**i for i in range(int(math.log2(2**20))+1)]

class CustomInput:
  def __init__(self, enunc = "", f = lambda *args: print (f"Empty function") ):
    self.table = widgets.Dropdown(
    options=Table,
    description='Tamanho da Tabela em Molduras',
    value=1,
)
    self.button = widgets.Button(description='Responder')
    self.f = f
    self.button.on_click(lambda *args: (f(self.table.value)))

  def display (self):
    display(self.table, self.button)

class CustomInput1:
  def __init__(self, enunc = "", f = lambda *args: print (f"Empty function") ):
    self.table = widgets.IntText(
    value=0,
    description='Digite um número inteiro:',
)
    self.button = widgets.Button(description='Responder')
    self.f = f
    self.button.on_click(lambda *args: (f(self.table.value)))

  def display (self):
    display(self.table, self.button)

class CustomInput2:
  def __init__(self, enunc = "", f = lambda *args: print (f"Empty function") ):
    self.table = widgets.IntText(
    value=0,
    description='Tab Ptr:',
    )
    self.data = widgets.IntText(
    value=0,
    description='Dados:',
    )
    self.button = widgets.Button(description='Responder')
    self.f = f
    self.button.on_click(lambda *args: (f(self.table.value,self.data.value)))

  def display (self):
    display(self.table, self.data, self.button)

class CustomInput3:
  def __init__(self, enunc = "", f = lambda *args: print (f"Empty function") ):
    self.table = widgets.Text(
    value="0",
    description='1 Address:',
    )
    self.data = widgets.Text(
    value="0",
    description='2 Address:',
    )
    self.button = widgets.Button(description='Responder')
    self.f = f
    self.button.on_click(lambda *args: (f(self.table.value,self.data.value)))

  def display (self):
    display(self.table, self.data, self.button)

In [ ]:
#@title Treinar as potências de 2
import ipywidgets as widgets
import math

# Create a dropdown widget
powers_of_2 = [2**i for i in range(int(math.log2(8192))+1)]
dropdown = widgets.Dropdown(
    options=powers_of_2,
    description='Power of 2:',
    value=1
)

# Define the function that computes log2
def compute_log2(power):
    result = math.log2(power)
    result = math.ceil(result)
    print(f"{power} requer {result} bits")

# Call the function using interact
widgets.interact(compute_log2, power=dropdown)


interactive(children=(Dropdown(description='Power of 2:', options=(1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024…

<function __main__.compute_log2(power)>

In [ ]:
from posixpath import pardir
#@title Gerador Aleatório

import math
import random

KB = 2**10
MB = 2**20
GB = 2**30


Virtual_size = [512*MB, 1*GB, 256*MB, 128*MB, 2*GB, 4*GB, 8*GB]
Physical_size = [64*KB, 128*KB, 256*KB, 8*MB, 16*MB, 32*MB, 64*MB]
page_size = [1*KB, 512, 2*KB, 8*KB, 4*KB, 2*MB,256]

def f(tam):
  if(tam % GB == 0):
    return f"{int(tam/GB)} GB"
  if(tam % MB == 0):
    return f"{int(tam/MB)} MB"
  if(tam % KB == 0):
    return f"{int(tam/KB)} KB"
  return str(tam)

def compute_pointer(a):
  bits = int(math.log2(a))
  if ( bits < 9):
    return 1
  elif ( bits < 17):
    return 2
  return 4



def gera_exec_pagina_1_nivel_tabela():
  res = None
  while True:
    idx = random.randrange(len(Virtual_size))
    v = Virtual_size[idx]
    idx = random.randrange(len(Physical_size))
    ph = Physical_size[idx]
    idx = random.randrange(len(page_size))
    p = page_size[idx]
    pointer_size = compute_pointer(ph/p)
    valid = (v/p)*pointer_size <= ph and ph > p
    table_size = v/p*pointer_size
    frames = math.ceil(table_size/p)
    res = {"size":frames}
    if (valid): break #avoid case that table size is bigger than required

  q = [v,ph,p]
  input = f"Calcule o tamanho da tabela de página em molduras para uma memória virtual de {f(v)} \n com memória física de {f(ph)} e página de {p} bytes"
  return input, res ,q

def endereco_1_nivel_tabela():
  res = None
  while True:
    idx = random.randrange(len(Virtual_size))
    v = Virtual_size[idx]
    idx = random.randrange(len(Physical_size))
    ph = Physical_size[idx]
    idx = random.randrange(len(page_size))
    p = page_size[idx]
    pointer_size = compute_pointer(ph/p)
    valid = (v/p)*pointer_size <= ph and ph > p and v > 64*p
    table_size = v/p*pointer_size
    frames = math.ceil(table_size/p)
    address = random.randrange(64*p)
    page = address//p*pointer_size
    offset = address%p
    data = frames*p+offset
    res = {"page":page,"data":data}
    if (valid): break #avoid case that table size is bigger than required

  q = [v,ph,p]
  input = f"Para uma memória virtual de {f(v)} com memória física de {f(ph)} e página de {p} bytes \n suponha que o primeiro acesso aloque a primeira moldura livre \n e que a tabela esteja no inicio da memoria \n quais os dois enderecos que o processador gera \n para acessar a {hex(address)}"
  return input, res ,q





def gera_exec_pagina_2_niveis_tabela():
  res = None
  while True:
    idx = random.randrange(len(Virtual_size))
    v = Virtual_size[idx]
    idx = random.randrange(len(Physical_size))
    ph = Physical_size[idx]
    idx = random.randrange(len(page_size))
    p = page_size[idx]
    pointer_size = compute_pointer(ph/p)
    valid = (v/p) > ph and v/p/p > 1 and (v/p/p*pointer_size) < ph
    first = v/p/p*pointer_size*pointer_size
    frames = math.ceil(first/p)+1
    res = {"size":frames}
    if (valid): break #avoid case that table size is bigger than required

  q = [v,ph,p]
  input = f"Calcule número mínimo de molduras ocupado pelos ponteiros da tabela de página \n considere apenas as páginas ocupadas por ponteiros do primeiro e segundo nivel \n para ter uma página de dados na memória \n com dois níveis de paginação, onde o segundo nível ocupa uma moldura, para uma memória virtual de {f(v)} \n com memória física de {f(ph)} e página de {p} bytes"
  return input, res ,q




Virtual2_size = [512*GB, 16*GB, 256*GB, 128*GB, 64*GB, 32*GB, 8*GB]
Physical2_size = [64*MB, 128*MB, 256*MB, 8*MB, 16*MB, 32*MB, 512*MB]
page2_size = [1*KB, 2*KB, 8*KB, 4*KB]

def Ocupacao_otima_1_5_moldura_pagina_2_niveis():
  res = None
  while True:
    idx = random.randrange(len(Virtual2_size))
    v = Virtual2_size[idx]
    idx = random.randrange(len(Physical2_size))
    ph = Physical2_size[idx]
    idx = random.randrange(len(page2_size))
    p = page2_size[idx]
    pointer_size = compute_pointer(ph/p)
    first = v/p/p*pointer_size*pointer_size
    valid = first < ph and  (pointer_size*1.5)*p < ph/p
    frames = math.ceil(first/p)+ math.ceil(pointer_size*1.5)
    res = {"size":frames}
    if (valid): break #avoid case that table size is bigger than required

  q = [v,ph,p]
  input = f"Calcule número mínimo de molduras ocupado pelos ponteiros da tabela de página \n para ter {1.5*p} páginas de dados na memória \n com dois níveis de paginação, onde o segundo nível ocupa uma moldura, para uma memória virtual de {f(v)} \n com memória física de {f(ph)} e página de {p} bytes"
  return input, res ,q


def fragment(x,y):
  if x < y:
    return y
  return x

def ocupacao_maxima_pagina_2_niveis_tabela():
  res = None
  while True:
    idx = random.randrange(len(Virtual2_size))
    v = Virtual2_size[idx]
    idx = random.randrange(len(Physical2_size))
    ph = Physical2_size[idx]
    idx = random.randrange(len(page2_size))
    p = page2_size[idx]
    pointer_size = compute_pointer(ph/p)
    valid = (v/p) > ph and v/p/p > 1
    first = v/p/p*pointer_size*pointer_size
    free_page = math.floor((ph - fragment(first,p))/p)
    complete = free_page//(1+p/pointer_size)
    remain = free_page%(1+p/pointer_size) -1
    if ( remain > 0 ):
      last = 1
    else:
      last = 0
    frames_L1 = math.ceil(fragment(first,p)/p)
    total = frames_L1+complete+last
    pages = p*complete/pointer_size+remain
    res = {"size":total, "pages": pages}
    if (valid): break #avoid case that table size is bigger than required


  q = [v,ph,p]
  input = f"Calcule número de molduras ocupadas pelos ponteiros da tabela de página e pelos dados \n incluindo primeiro e segundo nível \n para ter o máximo de páginas de dados na memória \n com mínimo de páginas de ponteiros de dois níveis de paginação, onde o segundo nível ocupa uma moldura, para uma memória virtual de {f(v)} \n com memória física de {f(ph)} e página de {p} bytes"
  return input, res ,q



# Exercicio 1

[video com dica para exercicio 1](https://youtu.be/rRuHeXdykSw) e desenho do excalidraw de [rascunho](https://excalidraw.com/#json=WKYq5KkImL7pxL3WbFszY,rEWRblPjVCN-DIq179YrgQ)

* [Exemplos da aula de revisão antes da prova para **QUESTÃO 3** da prova 2023](https://youtu.be/flxWyFYF6n0)

In [ ]:
#@title Exercicio 1 Tabela de Página com 1 nível, espaço ocupado em molduras
enunc, resposta, q = gera_exec_pagina_1_nivel_tabela()
Q1_start_time = getCurrentTime()
def avalia(*args):
  global Q1_start_time
  responseTime = getCurrentTime()
  if(resposta["size"] == args[0]):
    monitora("Q1_1nivel",[True,q],ID, Q1_start_time, responseTime)
    print("Correto")
  else:
    monitora("Q1_1nivel", [False,q], ID, Q1_start_time, responseTime)
    print("Falso")
  Q1_startTime = responseTime

print(enunc)
inp = CustomInput(f=avalia)
inp.display()


Calcule o tamanho da tabela de página em molduras para uma memória virtual de 128 MB 
 com memória física de 32 MB e página de 4096 bytes


Dropdown(description='Tamanho da Tabela em Molduras', options=(1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 20…

Button(description='Responder', style=ButtonStyle())

try to connect
17_24_1
Send `Q1_1nivel; [False, [134217728, 33554432, 4096]]; 71269; 17_29_54; 17_38_38; 17_24_1` to topic `inf450/2023/Colab_Virtual/71269/17_24_1`
Falso
try to connect
17_24_1
Send `Q1_1nivel; [False, [134217728, 33554432, 4096]]; 71269; 17_29_54; 17_38_42; 17_24_1` to topic `inf450/2023/Colab_Virtual/71269/17_24_1`
Falso
try to connect
17_24_1
Send `Q1_1nivel; [True, [134217728, 33554432, 4096]]; 71269; 17_29_54; 17_40_10; 17_24_1` to topic `inf450/2023/Colab_Virtual/71269/17_24_1`
Correto


# Exercicio 2

* [exemplo 128M virtual, 64KB fisica, pagina de 1024 - desenho](https://excalidraw.com/#json=wY3zItp8PzuSZi_Yz9P3K,iBH-0bYbXaK2y-aeSelNqg) + [explicacao em video neste exemplo ponteiro 1 byte](https://youtu.be/hCZrjIREtJU)

* [exemplo 128M virtual, 256KB fisica, pagina de 256, desenho](https://excalidraw.com/#json=Xhca7R9qzU9UdBdNtfjqY,qwCqUNL40u3NClJh9XVlRw) + [explicacao em video neste exemplo ponteiro 2 bytes](https://youtu.be/ezCwUkhQkLk)

* [Dois exemplos da aula de revisão para prova](https://youtu.be/AswXTFb_Xs0)


* [Exemplos da **QUESTÃO 4** da prova de 2023](https://youtu.be/MZrO_w3t4is)

In [ ]:
#@title **Exercicio 2** Tabela de Página com 2 niveis, espaço de uma página de dados, segundo nível usa 1 moldura
enunc, resposta, q = gera_exec_pagina_2_niveis_tabela()
Q2_start_time = getCurrentTime()
def avalia(*args):
  global Q2_start_time
  responseTime = getCurrentTime()
  if(resposta["size"] == args[0]):
    monitora("Q2_2nivel1pag",[True,q],ID, Q2_start_time, responseTime)
    print("Correto")
  else:
    monitora("Q2_2nivel1pag",[False,q],ID, Q2_start_time, responseTime)
    print("Falso")
  Q2_start_time = responseTime

print(enunc)
inp = CustomInput1(f=avalia)
inp.display()


Calcule número mínimo de molduras ocupado pelos ponteiros da tabela de página 
 considere apenas as páginas ocupadas por ponteiros do primeiro e segundo nivel 
 para ter uma página de dados na memória 
 com dois níveis de paginação, onde o segundo nível ocupa uma moldura, para uma memória virtual de 4 GB 
 com memória física de 256 KB e página de 256 bytes


IntText(value=0, description='Digite um número inteiro:')

Button(description='Responder', style=ButtonStyle())

try to connect
19_0_32
Send `Q2_2nivel1pag; [True, [4294967296, 262144, 256]]; 71269; 19_51_30; 19_54_18; 19_0_32` to topic `inf450/2023/Colab_Virtual/71269/19_0_32`
Correto


# Exercicio 3
* [Exemplo com virtual 16G com paginas de 2K, fisica 512M, ocupação ótima para 3072 páginas](https://excalidraw.com/#json=nVzzIXms8zQAT_7tBIyqh,RUQEQ-TnSorrcUfPMz2W7w) + [video com explicacoes com ponteiros de 4 bytes](https://youtu.be/L0kvdERnRv8)

* [Dois exemplos de aula de revisão para prova](https://youtu.be/0wk9WBrvpCA)


* [Exemplos da **QUESTÃO 4** da prova de 2023](https://youtu.be/MZrO_w3t4is)

In [ ]:
#@title **Exercicio 3** Tabela de Página com 2 niveis, espaço otimo para ocupação considerando X páginas, segundo nível usa 1 moldura
enunc, resposta, q = Ocupacao_otima_1_5_moldura_pagina_2_niveis()
Q3_start_time = getCurrentTime()
def avalia(*args):
  global Q3_start_time
  responseTime = getCurrentTime()
  if(resposta["size"] == args[0]):
    monitora("Q3_2nivelOtimoXpag",[True,q],ID, Q3_start_time, responseTime)
    print("Correto")
  else:
    monitora("Q3_2nivelOtimoXpag",[False,q],ID, Q3_start_time, responseTime)
    print("Falso")
  Q3_startTime = responseTime

print(enunc)
inp = CustomInput1(f=avalia)
inp.display()



Calcule número mínimo de molduras ocupado pelos ponteiros da tabela de página 
 para ter 1536.0 páginas de dados na memória 
 com dois níveis de paginação, onde o segundo nível ocupa uma moldura, para uma memória virtual de 128 GB 
 com memória física de 16 MB e página de 1024 bytes


IntText(value=0, description='Digite um número inteiro:')

Button(description='Responder', style=ButtonStyle())

try to connect
19_0_32
Send `Q3_2nivelOtimoXpag; [True, [137438953472, 16777216, 1024]]; 71269; 20_2_36; 20_7_59; 19_0_32` to topic `inf450/2023/Colab_Virtual/71269/19_0_32`
Correto


# Exercicio 4

* [Exemplo 256G virtual, 8M fisica, pagina 8k, ocupacao maxima e otima, segundo nivel 1 moldura](https://excalidraw.com/#json=T8fACajHVnseKtsfRB0JC,XwZp_GC0wvOqCGSicm4dlg) + [explicacoes com ponteiros de 2 bytes, não chega nem a ocupar uma moldura de segundo nível](https://youtu.be/RWm4ImJ18CY)

* [exemplo com 64 G Virtual, 32M fisica, 1k pagina, ocupacao maxima de otima da memoria com dados, segundo nivel 1 moldura](https://excalidraw.com/#json=zjhRtNucw_VvSzyjqlC6K,Fx5ZaooyZosPq0VVufFAEg) [explicacoes com 2 byte de ponteiros](https://youtu.be/1MPUm-YvBIg)

* [exemplo com 256 G Virtual, 64M fisica, 1k pagina, ocupacao maxima de otima da memoria com dados, segundo nivel 1 moldura](https://excalidraw.com/#json=2087Vs7HCWfqAxe7qFCPZ,TlWOIBM3UClCFTc6-y44Dg) [explicacoes com 2 byte de ponteiros](https://youtu.be/EogoROU_atA)

* [Dois exemplos da aula de revisão para prova](https://youtu.be/8D6M0YOWrQQ)

* [Exemplos para **QUESTÃO 5** da prova 2023](https://youtu.be/RIRGim_mpjQ)

In [ ]:
#@title **Exercicio 4** Tabela de Página com 2 niveis, espaço otimo para ocupação maxima, segundo nível usa 1 moldura
enunc, resposta, q = ocupacao_maxima_pagina_2_niveis_tabela()
Q4_start_time = getCurrentTime()
def avalia(*args):
  global Q4_start_time
  responseTime = getCurrentTime()
  if(resposta["size"] == args[0] and resposta["pages"] == args[1]):
    monitora("Q4_2NiveisOtimoCheia",[True,q],ID, Q4_start_time, responseTime)
    print("Correto")
  else:
    monitora("Q4_2NiveisOtimoCheia",[False,q],ID, Q4_start_time, responseTime)
    print("Falso")
  Q4_startTime = responseTime

print(enunc)
inp = CustomInput2(f=avalia)
inp.display()



Calcule número de molduras ocupadas pelos ponteiros da tabela de página e pelos dados 
 incluindo primeiro e segundo nível 
 para ter o máximo de páginas de dados na memória 
 com mínimo de páginas de ponteiros de dois níveis de paginação, onde o segundo nível ocupa uma moldura, para uma memória virtual de 512 GB 
 com memória física de 8 MB e página de 1024 bytes


IntText(value=0, description='Tab Ptr:')

IntText(value=0, description='Dados:')

Button(description='Responder', style=ButtonStyle())

try to connect
19_0_32
Send `Q4_2NiveisOtimoCheia; [False, [549755813888, 8388608, 1024]]; 71269; 20_17_40; 20_26_33; 19_0_32` to topic `inf450/2023/Colab_Virtual/71269/19_0_32`
Falso
try to connect
19_0_32
Send `Q4_2NiveisOtimoCheia; [True, [549755813888, 8388608, 1024]]; 71269; 20_17_40; 20_26_38; 19_0_32` to topic `inf450/2023/Colab_Virtual/71269/19_0_32`
Correto


# exercicio 5

* [esboco para 256M virtual, 64M fisica, pagina de 512, ponteiro de 4 bytes, ocupando primeira moldura livre para endereco 0xea6](https://excalidraw.com/#json=B4nuEowviPqVOMzKFLI97,a3A68O1_YjtpRZyHp077-A) + [video com explicações](https://youtu.be/S8qE2lUfsgM)

* [esboco para 512M virtual, 128K fisica, 8K pagina, ponteiro 1 byte](https://excalidraw.com/#json=ECxMRRzjVzXYQMMsCAqAg,6nx93GihiE1I17BfUxjZPg) + [video com explicacoes](https://youtu.be/LuTFS1W2s_k)

* [esboco para 2G virtual, 8M fisica, 4K pagina, ponteiro 2 bytes](https://excalidraw.com/#json=BZXXXTKJv1G_TKbFoQ2FL,gM_kcSKm0M6GdLc9AeXAZA) + [video com explicacoes](https://youtu.be/B7zqUHl5uWo)

* [dois exemplos da aula de revisão para prova](https://youtu.be/KyUlZ2BkSOQ)

* [Exemplos da aula de revisão antes da prova para **QUESTÃO 3** da prova 2023](https://youtu.be/flxWyFYF6n0)

In [ ]:
#@title **Exercicio 5** Tabela de Página com 1 nivel, ocupa primeira moldura livre, quais são os dois acessos ?
enunc, resposta, q = endereco_1_nivel_tabela()
Q5_start_time = getCurrentTime()
def avalia(*args):
  global Q5_start_time
  responseTime = getCurrentTime()
  if(resposta["page"] == int(args[0],16) and resposta["data"] == int(args[1],16)):
    monitora("Q5_1nivel_mapeamento",[True,q],ID, Q5_start_time, responseTime)
    print("Correto")
  else:
    monitora("Q5_1nivel_mapeamento",[False,q],ID, Q5_start_time, responseTime)
    print("Falso")
  Q5_startTime = responseTime

print(enunc)
inp = CustomInput3(f=avalia)
inp.display()



Para uma memória virtual de 8 GB com memória física de 64 MB e página de 2097152 bytes 
 suponha que o primeiro acesso aloque a primeira moldura livre 
 e que a tabela esteja no inicio da memoria 
 quais os dois enderecos que o processador gera 
 para acessar a 0x64123e7


Text(value='0', description='1 Address:')

Text(value='0', description='2 Address:')

Button(description='Responder', style=ButtonStyle())

try to connect
19_0_32
Send `Q5_1nivel_mapeamento; [True, [8589934592, 67108864, 2097152]]; 71269; 19_31_39; 19_39_27; 19_0_32` to topic `inf450/2023/Colab_Virtual/71269/19_0_32`
Correto
